# Iteracyjne metody rozwiązywania równań liniowych

### Czytanka
* Kincaid, Cheney, rozdz. 8.2, str. 319 (bardzo przystępnie napisane)
* Normy wektorów i macierzy:
    * wektorowa: https://en.wikipedia.org/wiki/Norm_(mathematics)
    * macierzowa: https://en.wikipedia.org/wiki/Matrix_norm
* Metoda Jacobiego: https://en.wikipedia.org/wiki/Jacobi_method
* Metoda SOR: https://en.wikipedia.org/wiki/Successive_over-relaxation
* Metoda Gaussa-Seidela: https://en.wikipedia.org/wiki/Gauss%E2%80%93Seidel_method
* (ale wystarczy K&C)

### Troszkę teorii

Chcemy rozwiązać układ równań liniowych postaci $A\mathbf {x} =\mathbf {b} $, gdzie:

$$
A={\begin{bmatrix}a_{11}&a_{12}&\cdots &a_{1n}\\a_{21}&a_{22}&\cdots &a_{2n}\\\vdots &\vdots &\ddots &\vdots \\a_{n1}&a_{n2}&\cdots &a_{nn}\end{bmatrix}},\qquad \mathbf {x} ={\begin{bmatrix}x_{1}\\x_{2}\\\vdots \\x_{n}\end{bmatrix}},\qquad \mathbf {b} ={\begin{bmatrix}b_{1}\\b_{2}\\\vdots \\b_{n}\end{bmatrix}}.
$$

Mimo, że dobrze znamy dokładne metody rozwiązania takiego równania, częstokroć w praktyce nie możemy ich zastosować -- przede wszystkim ze względu na rozmiar problemu. Rozwiązanie? Zastosować metody iteracyjne, które, choć nie dadzą nam dokładnego wyniku, pozwolą nam w rozsądnym czasie uzyskać dobrą aproksymację. (Zresztą, dokładne metody też nie dają dokładnych rezultatów z powodu błędów arytmetyki zmiennoprzecinkowej).

_Suchy żarcik dnia: John ma problem. John myśli: "Wiem, użyję arytmetyki zmiennoprzecinkowej." Teraz John ma 1.999999997 problemu. (Zasłyszane w pracy)._


#### Metoda Jacobiego

Metody Jacobiego możemy użyć pod warunkiem, że macierz jest przekątniowo dominująca, tj. mamy $ |a_{ii}|\geq \sum _{j\neq i}|a_{ij}|\quad {\text{dla każdego }}i. $

Pomysł polega na rozkładzie macierzy A na **sumę** dwóch macierzy:
$$
A=D+R\qquad {\text{gdzie}}\qquad D={\begin{bmatrix}a_{11}&0&\cdots &0\\0&a_{22}&\cdots &0\\\vdots &\vdots &\ddots &\vdots \\0&0&\cdots &a_{nn}\end{bmatrix}}{\text{ oraz }}R={\begin{bmatrix}0&a_{12}&\cdots &a_{1n}\\a_{21}&0&\cdots &a_{2n}\\\vdots &\vdots &\ddots &\vdots \\a_{n1}&a_{n2}&\cdots &0\end{bmatrix}}.
$$

Następnie krok iteracji wygląda następująco:
$$ \mathbf {x} ^{(k+1)}=D^{-1}(\mathbf {b} -R\mathbf {x} ^{(k)}), $$

I element po elemencie:
$$ x_{i}^{(k+1)}={\frac {1}{a_{ii}}}\left(b_{i}-\sum _{j\neq i}a_{ij}x_{j}^{(k)}\right),\quad i=1,2,\ldots ,n. $$

Zwróćmy uwagę, że cały trick polega na tym, że macierz $D$ bardzo łatwo odwrócić.

### Zadanie 1.
Zaimplementować metodę Jacobiego i przetestować jej działanie na paru układach równań. Porównać z metodą eliminacji Gaussa użytą do tych samych równań.

In [1]:
import numpy as np

def jacobi_solve(A: np.matrix, b: np.matrix, iterations = 1000) -> np.matrix:
    n = A.shape[0]
    x = np.zeros(n)
    k = 0
    while k < iterations:
        x1 = np.zeros(n)
        for i in range(n):
            sum = 0
            for j in range(n):
                if j != i:
                    sum += A[i,j] * x[j]
            x1[i] = (1 / A[i,i]) * (b[i] - sum)
        x = x1
        k = k + 1
    return x

#### Metoda Gaussa-Seidela

Opiera się na tym samym pomyśle, co metoda Jacobiego, ale przy innym rozkładzie macierzy $A$:

$$
A=L_{*}+U\qquad {\text{where}}\qquad L_{*}={\begin{bmatrix}a_{11}&0&\cdots &0\\a_{21}&a_{22}&\cdots &0\\\vdots &\vdots &\ddots &\vdots \\a_{n1}&a_{n2}&\cdots &a_{nn}\end{bmatrix}},\quad U={\begin{bmatrix}0&a_{12}&\cdots &a_{1n}\\0&0&\cdots &a_{2n}\\\vdots &\vdots &\ddots &\vdots \\0&0&\cdots &0\end{bmatrix}}.
$$

Wtedy układ równań możemy zapisać jako: $ L_{*}\mathbf {x} =\mathbf {b} -U\mathbf {x} $ i iterować tak:

$$ \mathbf {x} ^{(k+1)}=L_{*}^{-1}(\mathbf {b} -U\mathbf {x} ^{(k)}). $$

Element po elemencie:

$$ {\displaystyle x_{i}^{(k+1)}={\frac {1}{a_{ii}}}\left(b_{i}-\sum _{j=1}^{i-1}a_{ij}x_{j}^{(k+1)}-\sum _{j=i+1}^{n}a_{ij}x_{j}^{(k)}\right),\quad i=1,2,\dots ,n.} $$

Podobnie jak z Jacobim, tutaj trick polega na tym, że macierz L jest łatwa do odwrócenia.

### Zadanie 2.
Zaimplementować metodę Gaussa-Seidela i przetestować na tych samych układach równań, co metodę Jacobiego. 

In [2]:
def gauss_seidel_solve(A: np.matrix, b: np.matrix, iterations = 1000) -> np.matrix:
    n = A.shape[0]
    x = np.zeros(n)
    for it_count in range(iterations):
        x_new = np.zeros_like(x)

        for i in range(A.shape[0]):
            s1 = np.dot(A[i, :i], x_new[:i])
            s2 = np.dot(A[i, i + 1:], x[i + 1:])
            x_new[i] = (b[i] - s1 - s2) / A[i, i]
        x = x_new
    return x

In [3]:
A = np.matrix([[3,2],
               [4,6]])
b = np.matrix([9,14]).transpose()

In [4]:
jacobi_solve(A, b)

array([2.6, 0.6])

In [5]:
gauss_seidel_solve(A, b)

array([2.6, 0.6])

#### Metoda SOR (Successive Over Relaxation)

Znowu podobnie, tyle, że tym razem rozkładamy macierz na sumę trzech macierzy:

$$
D={\begin{bmatrix}a_{11}&0&\cdots &0\\0&a_{22}&\cdots &0\\\vdots &\vdots &\ddots &\vdots \\0&0&\cdots &a_{nn}\end{bmatrix}},\quad L={\begin{bmatrix}0&0&\cdots &0\\a_{21}&0&\cdots &0\\\vdots &\vdots &\ddots &\vdots \\a_{n1}&a_{n2}&\cdots &0\end{bmatrix}},\quad U={\begin{bmatrix}0&a_{12}&\cdots &a_{1n}\\0&0&\cdots &a_{2n}\\\vdots &\vdots &\ddots &\vdots \\0&0&\cdots &0\end{bmatrix}}.
$$

Co umożliwia zapisanie układu równań tak: $ (D+\omega L)\mathbf {x} =\omega \mathbf {b} -[\omega U+(\omega -1)D]\mathbf {x} $ i daje następujące wzory na iterację:

$$ \mathbf {x} ^{(k+1)}=(D+\omega L)^{-1}{\big (}\omega \mathbf {b} -[\omega U+(\omega -1)D]\mathbf {x} ^{(k)}{\big )}=L_{w}\mathbf {x} ^{(k)}+\mathbf {c} , $$

oraz:

$$ x_{i}^{(k+1)}=(1-\omega )x_{i}^{(k)}+{\frac {\omega }{a_{ii}}}\left(b_{i}-\sum _{j<i}a_{ij}x_{j}^{(k+1)}-\sum _{j>i}a_{ij}x_{j}^{(k)}\right),\quad i=1,2,\ldots ,n. $$

### Zadanie 3.
Zaimplementować metodę SOR i przetestować na tych samych układach.

In [6]:
def sor_solve(A: np.matrix, b: np.matrix, iterations = 1000) -> np.matrix:
    n = A.shape[0]
    x = np.zeros(n)
    omega = 1.44
    k = 0
    while k < iterations:
        x1 = np.zeros(n)
        for i in range(n):
            sumL = 0
            sumU = 0
            for j in range(0, i):
                sumL = sumL + A[i,j] * x1[j]
            for j in range(i+1, n):
                sumU = sumU + A[i,j] * x[j]
            x1[i] = (1 - omega) * x[i] + (omega / A[i,i]) * (b[i] - sumL - sumU)
        x = x1
        k = k + 1
    return x

In [7]:
sor_solve(A, b)

array([2.6, 0.6])

### Zadanie 4.
Dla powyższych metod porównać (na wykresie) tempo zbiegania do rozwiązania.

### Pytanie
Jakie jest kryterium zbieżności metod powyżej? Czy zawsze można je stosować?

#### Bonus:
Jak przeklejać piękne wzory z Wikipedii i się przy tym nie namęczyć? (na zajęciach).

In [8]:
def check(A: np.matrix, b: np.matrix, method):
    xt = np.linalg.solve(A, b)
    for i in range(1,100):
        x = method(A, b, i)
        if(np.allclose(xt.transpose(), x)):
            print(method.__name__,"- zbieżność po", i, "iteracjach")
            break

In [9]:
check(A, b, sor_solve)
check(A, b, gauss_seidel_solve)
check(A, b, jacobi_solve)

sor_solve - zbieżność po 16 iteracjach
gauss_seidel_solve - zbieżność po 15 iteracjach
jacobi_solve - zbieżność po 30 iteracjach


In [ ]:
Metody Gaussa-Seidla oraz Jacobiego są zbieżne, 
jeśli macierz A spełnia warunek ścisłej dominacji przekątniowej w rzędach lub kolumnach.